In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This dataset contains the fundamentals of a series of Tickets over 16 years. Each row is one `Ticker` and the `date` of the fundamental. The column `freq` is `Q` or `A` depending if that fundamental is for a quarter or for a year, respectively.

In [4]:
data = pd.read_feather("data/financials_against_sp500.feather")
data.tail()

,Ticker,date,freq,Asset Turnover,Basic EPS,Basic Shares Outstanding,Book Value Per Share,Cash Flow From Financial Activities,Cash Flow From Investing Activities,Cash Flow From Operating Activities,...,dividend_cumsum_in_period_-730,stock_change_div_-730,sp500_change_-730,improve_sp500,std_365,std_730,std_-120,std_-365,std_-730,sharpe_ratio_365
277053,WOR,2021-11-30,Q,NaN,2.19,50.381,NaN,-32.85,-22.098,-119.104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277054,ADBE,2021-11-30,A,NaN,10.10,477.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277055,ADBE,2021-11-30,Q,NaN,2.59,476.000,NaN,3223.00,1798.000,-5164.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277056,COST,2021-11-30,Q,0.7851,2.99,443.377,42.8474,-839.00,-912.000,3258.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277057,ORCL,2021-11-30,Q,0.0969,-0.46,2694.000,-3.6159,-12053.00,10730.000,-3682.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data[['Ticker', 'execution_date', 'date']]

,Ticker,execution_date,date
0,A,2005-06-30,2005-01-31
1,CNR,2005-06-30,2005-01-31
2,DY,2005-06-30,2005-01-31
3,GYRO,2005-06-30,2005-01-31
4,CULP,2005-06-30,2005-01-31
...,...,...,...
277053,WOR,2022-03-31,2021-11-30
277054,ADBE,2022-03-31,2021-11-30
277055,ADBE,2022-03-31,2021-11-30
277056,COST,2022-03-31,2021-11-30


Create a column called `period` with the quarter or the year, depending of `freq`:

In [3]:
data.loc[data["freq"]=="Q","period"] = data["date"].dt.year.astype(str) + "_" + data["date"].dt.quarter.astype(str)
data.loc[data["freq"]=="A","period"] = data["date"].dt.year.astype(str)
# data = data.drop(columns = ["date"])
# ensure there is only one row per Ticker and period
print(data.groupby(["Ticker","freq","period"]).size().value_counts())

1    275593
2       731
3         1
dtype: int64


In [4]:
print(data.shape)
data = data.sort_values(["Ticker","freq","period"]).drop_duplicates(["Ticker","freq","period"])
print(data.shape)

(277058, 141)
(276325, 141)


Ensure that we have 1 and only 1 row for each combination of `period`, `freq` and `Ticker`:

In [5]:
all_period_quarter = data[data["freq"]=="Q"].sort_values("period")["period"].unique()
all_period_anual = data[data["freq"]=="A"].sort_values("period")["period"].unique()
all_periods = pd.concat([pd.DataFrame({"period":all_period_quarter,"freq":"Q"}),pd.DataFrame({"period":all_period_anual,"freq":"A"})])
all_periods = all_periods.merge(data.drop_duplicates("Ticker")["Ticker"], how='cross')
data = pd.merge(all_periods,data,on = ["freq","period","Ticker"],how = "left")
# ensure that for each ticker and frequency we have the same number of rows
assert len(data[data["freq"]=="Q"].groupby(["Ticker","freq"]).size().reset_index()[0].value_counts()) ==1
assert len(data[data["freq"]=="A"].groupby(["Ticker","freq"]).size().reset_index()[0].value_counts()) ==1


Compute the Market Cap:

Put NaN on negative values of `Shares Outstanding`:

In [6]:
data.loc[data['Shares Outstanding'] <= 0, 'Shares Outstanding'] = np.nan

In [7]:
data = data.sort_values(["Ticker","period"],ascending = True)
data['Filled Shares Outstanding'] = data.groupby('Ticker')['Shares Outstanding'].fillna(method = 'ffill')
data['Market_cap'] = data['close_0'] * data['Filled Shares Outstanding']

In [8]:
data.reset_index().to_feather("data/trns_financials_against_sp500.feather")